In [1]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

data = fetch_california_housing()
feature_names = data.feature_names
feature_names


['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [2]:
X, y = data.data, data.target

# only use first 10 samples for training
num_sample = 10
X, y = X[:num_sample], y[:num_sample]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print('X_train.shape: ',X_train.shape)
print('X_test.shape: ',X_test.shape)

X_train.shape:  (7, 8)
X_test.shape:  (3, 8)


In [8]:
# normalize the features
def standardize(X_train, X_test):
    """
    Standardize features by removing the mean and scaling to unit variance.

    The standard score of a sample x is calculated as:
        z = (x - mean) / std

    where mean and std are computed from the training set.

    Args:
        X_train: Training data of shape (n_samples, n_features)
        X_test: Test data of shape (m_samples, n_features)

    Returns:
        X_train_standardized: Standardized training data
        X_test_standardized: Standardized test data (using training statistics)
    """
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)

    mean = np.nanmean(X_train, axis=0)
    std = np.nanstd(X_train, axis=0, ddof=0)

    # avoid divide-by-zero: leave zero-variance columns at 0 after centering
    std_safe = std.copy()
    std_safe[std_safe == 0] = 1.0

    X_train_standardized = (X_train - mean) / std_safe
    X_test_standardized = (X_test - mean) / std_safe

    return X_train_standardized, X_test_standardized

X_train, X_test = standardize(X_train, X_test)


In [10]:

alphas = np.logspace(-3, 3, 50)

lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42, max_iter=10000)
lasso_cv.fit(X_train, y_train)

best_alpha = lasso_cv.alpha_
y_pred = lasso_cv.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
n_surviving_features = np.sum(lasso_cv.coef_ != 0)

print(f"Best α: {best_alpha:.6f}")
print(f"Surviving features: {n_surviving_features}")
print(f"Test MSE: {test_mse:.4f}")
print("Surviving feature names:", [f for f, c in zip(feature_names, lasso_cv.coef_) if c != 0])

# Plot regularization path and MSE analysis
print("\n" + "="*60)
print("Regularization Path Analysis")
print("="*60)


Best α: 0.051795
Surviving features: 4
Test MSE: 0.3660
Surviving feature names: ['AveBedrms', 'Population', 'AveOccup', 'Latitude']

Regularization Path Analysis


In [12]:

# Calculate coefficients and errors for each alpha
coef_path = []
train_errors = []
for alpha in alphas:
    lasso = Lasso(alpha=alpha, max_iter=10000)
    lasso.fit(X_train, y_train)
    coef_path.append(lasso.coef_)
    y_train_pred = lasso.predict(X_train)
    train_errors.append(mean_squared_error(y_train, y_train_pred))

coef_path = np.array(coef_path)  # shape: (n_alphas, n_features)

# Get cross-validation results
# LassoCV stores the alphas it actually used
cv_alphas = lasso_cv.alphas_
mse_path = lasso_cv.mse_path_  # shape: (n_alphas, n_folds)
mean_mse = np.mean(mse_path, axis=1)
std_mse = np.std(mse_path, axis=1)



NameError: name 'lasso' is not defined

In [6]:
# ========== Plot 1: MSE vs Alpha ==========
fig2, ax2 = plt.subplots(figsize=(10, 6))

# Plot training error (using our alphas)
ax2.plot(alphas, train_errors, 'b-', linewidth=2.5, label='Training MSE', alpha=0.8)

# Plot validation error with confidence interval (using CV alphas)
ax2.plot(cv_alphas, mean_mse, 'r-', linewidth=2.5, label='Validation MSE (5-Fold CV)', alpha=0.8)

# Mark the best alpha - find the minimum validation MSE
best_idx = np.argmin(mean_mse)
best_val_mse = mean_mse[best_idx]
ax2.axvline(x=best_alpha, color='green', linestyle='--', linewidth=2,
            label=f'Best α = {best_alpha:.4f}')
ax2.plot(best_alpha, best_val_mse, 'go', markersize=10,
         label=f'Min Val MSE = {best_val_mse:.4f}', zorder=5)

# Formatting
ax2.set_xscale('log')
ax2.set_xlabel('λ', fontsize=14, fontweight='bold')
ax2.set_ylabel('Mean Squared Error', fontsize=12, fontweight='bold')
ax2.set_title('MSE vs. α', fontsize=16, fontweight='bold', pad=15)
ax2.legend(loc='best', fontsize=9, framealpha=0.9)
ax2.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()
print(f"\nFeature names: {feature_names}")
print(f"\nInterpretation:")
print(f"- At optimal α = {best_alpha:.4f}, {n_surviving_features} features survive")
print(f"- This balances model complexity and generalization performance")
print(f"- Using K=5 folds for cross-validation with small dataset (n={len(X_train)})")

NameError: name 'plt' is not defined

In [7]:
# ========== Plot 2: Regularization Path ==========
fig1, ax1 = plt.subplots(figsize=(10, 6))
colors = plt.cm.tab10(np.linspace(0, 1, len(feature_names)))
for i, (feature_name, color) in enumerate(zip(feature_names, colors)):
    ax1.plot(alphas, coef_path[:, i], linewidth=2.5, label=feature_name, color=color)

# Mark the best alpha with vertical line
ax1.axvline(x=best_alpha, color='black', linestyle='--', linewidth=2,
            label=f'Best α = {best_alpha:.4f}')

# Formatting
ax1.set_xscale('log')
ax1.set_xlabel('λ', fontsize=14, fontweight='bold')
ax1.set_ylabel('Coefficient Value', fontsize=12, fontweight='bold')
ax1.set_title('Regularization Path', fontsize=16, fontweight='bold', pad=15)
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.8, alpha=0.3)
ax1.legend(loc='best', fontsize=9, framealpha=0.9)
ax1.grid(True, alpha=0.2, linestyle='--')

plt.tight_layout()
plt.show()
print(f"\nFeature names: {feature_names}")
print(f"\nInterpretation:")
print(f"- At optimal α = {best_alpha:.4f}, {n_surviving_features} features survive")
print(f"- This balances model complexity and generalization performance")
print(f"- Using K=5 folds for cross-validation with small dataset (n={len(X_train)})")


NameError: name 'plt' is not defined